In [3]:
!pip install mem0ai langchain_openai langchain_core python-dotenv -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.4/266.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.

Store OPENAI_API_KEY in .env

In [4]:
import dotenv
dotenv.load_dotenv()

False

In [10]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [11]:
from mem0 import Memory
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [12]:
custom_prompt = """
Please only extract entities containing patient health information, appointment details, and user information.
Here are some few shot examples:

Input: Hi.
Output: {{"facts" : []}}

Input: The weather is nice today.
Output: {{"facts" : []}}

Input: I have a headache and would like to schedule an appointment.
Output: {{"facts" : ["Patient reports headache", "Wants to schedule an appointment"]}}

Input: My name is Jane Smith, and I need to reschedule my appointment for next Tuesday.
Output: {{"facts" : ["Patient name: Jane Smith", "Wants to reschedule appointment", "Original appointment: next Tuesday"]}}

Input: I have diabetes and my blood sugar is high.
Output: {{"facts" : ["Patient has diabetes", "Reports high blood sugar"]}}

Return the facts and patient information in a json format as shown above.
"""

In [13]:
config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4o",
            "temperature": 0.1,
            "max_tokens": 2000,
        }
    },
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-large"
        }
    },
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "test-123",
            "embedding_model_dims": 3072,
        }
    },
    "custom_prompt": custom_prompt,
    "version": "v1.1",
}

In [14]:
class AIHealthcareSupport:
    def __init__(self, config):
        """
        Initialize the AI Healthcare Support with Memory Configuration and Langchain OpenAI Chat Model.

        :param config: Configuration for memory and model settings.
        """
        self.memory = Memory.from_config(config)
        self.app_id = "app-1"
        self.model = ChatOpenAI(model="gpt-4o")

    def ask(self, question, user_id=None):
        """
        Ask a question to the AI and store the relevant facts in memory.

        :param question: The question to ask the AI.
        :param user_id: Optional user ID to associate with the memory.
        :return: Response from the AI along with the original question.
        """
        # Retrieve relevant memories using the search_memory method
        memories = self.search_memory(question, user_id=user_id)

        context = "Relevant information from previous conversations:\n"
        if memories['results']:
            for memory in memories['results']:
                context += f" - {memory['memory']}\n"
        print(context)

        messages = [
            SystemMessage(content=f"""You are a helpful healthcare support assistant. Use the provided context to personalize your responses and remember user health information and past interactions. {context}"""),
            HumanMessage(content=question)
        ]

        response = self.model.invoke(messages)

        # Store the interaction in memory
        self.add_memory(question, response.content, user_id=user_id)
        return {"messages": [response.content]}

    def add_memory(self, question, response, user_id=None):
        """
        Add a memory entry to the memory store.

        :param question: The question that was asked by the user.
        :param response: The response generated by the AI.
        :param user_id: Optional user ID to associate with the memory.
        """
        self.memory.add(f"User: {question}\nAssistant: {response}", user_id=user_id, metadata={"app_id": self.app_id})

    def get_memories(self, user_id=None):
        """
        Retrieve all memories associated with the given user ID.

        :param user_id: Optional user ID to filter memories.
        :return: List of memories.
        """
        return self.memory.get_all(user_id=user_id)

    def search_memory(self, query, user_id=None):
        """
        Search for memories related to the given query and user ID.

        :param query: The query to search for in the memories.
        :param user_id: Optional user ID to filter memories.
        :return: List of relevant memories.
        """
        related_memories = self.memory.search(query, user_id=user_id)
        return related_memories

In [15]:
# Initialize the AIHealthcareSupport bot
ai_support = AIHealthcareSupport(config)
# User ID for interaction
user_id = "James"

In [16]:
# Interacting with the bot
print("Interacting with AI Healthcare Support:\n")

# Example interactions
questions = [
    "I have a family history of diabetes; how can I reduce my risk?",  # Preventive care inquiry
    "Can you recommend any specific dietary changes?",  # Focusing on diet
    "I have a headache and would like to schedule an appointment."
]

# Loop through each question, ask the bot, and print responses
for question in questions:
    response = ai_support.ask(question, user_id=user_id)
    print(f"User: {question}")
    print(f"AI: {response['messages'][0]}\n")

Interacting with AI Healthcare Support:

Relevant information from previous conversations:

User: I have a family history of diabetes; how can I reduce my risk?
AI: Reducing your risk of developing diabetes, especially with a family history, involves making some lifestyle changes and monitoring your health. Here are a few steps you can take:

1. **Maintain a Healthy Weight:** Achieving and maintaining a healthy weight can significantly reduce your risk. Even a small amount of weight loss can have a positive impact.

2. **Eat a Balanced Diet:** Focus on a diet rich in whole foods, including plenty of vegetables, fruits, whole grains, and lean proteins. Try to limit processed foods, added sugars, and unhealthy fats.

3. **Stay Active:** Aim for at least 150 minutes of moderate aerobic activity, such as brisk walking, or 75 minutes of vigorous activity, such as running, each week. Also, include strength training exercises twice a week.

4. **Regular Check-Ups:** Keep up with regular healt

In [17]:
# Retrieve and display memories associated with the user
memories = ai_support.get_memories(user_id=user_id)
print("All Memories:")
for memory in memories['results']:
    print(f"- {memory['memory']}")

All Memories:
- Patient has a family history of diabetes
- Patient reports headache
- Wants to schedule an appointment
- Wants to know how to reduce risk of diabetes


In [18]:
question = "I've been experiencing some chest pain; what should I do?"
response = ai_support.ask(question, user_id="john")
print(f"User: {question}")
print(f"AI: {response['messages'][0]}\n")

Relevant information from previous conversations:

User: I've been experiencing some chest pain; what should I do?
AI: I'm sorry to hear you're experiencing chest pain. It's important to take this seriously. Chest pain can have various causes, some of which require immediate medical attention. Here are some steps you should consider:

1. **Seek Immediate Medical Help:** If the chest pain is severe, feels like pressure or squeezing, or is accompanied by other symptoms like shortness of breath, sweating, nausea, or pain radiating to the arm or jaw, call emergency services right away.

2. **Rest and Stay Calm:** Try to sit down and relax. Avoid any physical exertion until you have a better understanding of what's causing the pain.

3. **Consider Other Symptoms:** Pay attention to any other symptoms you might be experiencing, like dizziness, fainting, or palpitations, and share these with a healthcare provider.

4. **Medical History:** Think about your personal and family medical history. 

In [19]:
memories = ai_support.get_memories(user_id=user_id)
memories

{'results': [{'id': '1b906bba-b3ee-4b6f-8c65-b232c6502d5e',
   'memory': 'Patient has a family history of diabetes',
   'hash': '9c93d140616969698c42f0a6cc6f0df2',
   'metadata': {'app_id': 'app-1'},
   'created_at': '2024-10-09T01:14:15.692263-07:00',
   'updated_at': None,
   'user_id': 'James'},
  {'id': '42770842-adc7-4a59-a483-5023657a0e25',
   'memory': 'Patient reports headache',
   'hash': 'a1e1e89176ff726dfb8c218fff6d8491',
   'metadata': {'app_id': 'app-1'},
   'created_at': '2024-10-09T01:14:48.917910-07:00',
   'updated_at': None,
   'user_id': 'James'},
  {'id': 'c72730fa-9989-41bc-bf50-404feca48725',
   'memory': 'Wants to schedule an appointment',
   'hash': '1076cd460c182e6aa3fb1c830242192c',
   'metadata': {'app_id': 'app-1'},
   'created_at': '2024-10-09T01:14:48.939844-07:00',
   'updated_at': None,
   'user_id': 'James'},
  {'id': 'e6c6dc34-7c0e-4803-9985-62ad59549bfd',
   'memory': 'Wants to know how to reduce risk of diabetes',
   'hash': '89ee57fd30f0f53a14129a24